In [29]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import re
from datetime import timedelta

In [30]:
data_list = []

def no_space(text):
    text1 = re.sub('&nbsp; | &nbsp;|\n|\t|\r| \n| \t| \r|\n |\t |\r ', '', text)
    text2 = re.sub('\n', '', text1)
    return text2

for page in range(1,3):
    url = 'https://pann.nate.com/talk/c20002?page='+ str(page)
#     print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.select(".subject > h2 > a")
    for article in articles:
        arti_url = article["href"]
        article_response = requests.get('https://pann.nate.com' + arti_url)
        article_soup = BeautifulSoup(article_response.content, "html.parser")
        
        title = article_soup.select_one(".post-tit-info > h1").text.strip()   #title
        contents_b = article_soup.select_one("#contentArea").text   #contents 전처리 전
        contents = no_space(contents_b)   #contents 전처리 완
        view_cnt = article_soup.select_one('.count').text   #조회수
        postdate_txt = article_soup.select_one(".date").text   #업로드일 전처리 1
        postdate_tx = datetime.strptime(postdate_txt, '%Y.%m.%d %H:%M')   #업로드일 전처리 2
        postdate = postdate_tx.strftime('%Y-%m-%d')   #업로드일 전처리 완
        review_cnt = article_soup.select_one(".num > strong").text   #리뷰수
        
        data_list.append([title,contents,view_cnt[2:],review_cnt,postdate])

In [31]:
with open("nate_talk_20_page2_230705.csv", "w", encoding = "utf-8", newline="") as file:   #csv파일로 변환
    writer = csv.writer(file)
    writer.writerow(["title", "contents", "view_cnt", "review_cnt", "postdate"])
    writer.writerows(data_list)